In [3]:
import pandas as pd

df = pd.read_csv("C:/Users/saisu/Documents/Learning/RAG_project_imdb/data/imdb_top_1000.csv")
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


In [2]:
df_experiment=df.head()
df_experiment.to_csv("imdb_dataset.csv")

In [2]:
df.columns

Index(['Poster_Link', 'Series_Title', 'Released_Year', 'Certificate',
       'Runtime', 'Genre', 'IMDB_Rating', 'Overview', 'Meta_score', 'Director',
       'Star1', 'Star2', 'Star3', 'Star4', 'No_of_Votes', 'Gross'],
      dtype='object')

In [3]:
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document

documents = [Document(metadata={"title": row["Series_Title"]}, 
    page_content=f"Released Year: {row['Released_Year']}, Certificate: {row['Certificate']}, Runtime: {row['Runtime']}, Genre: {row['Genre']}, IMDB Rating: {row['IMDB_Rating']}, Overview: {row['Overview']}, Meta score: {row['Meta_score']}, Director: {row['Director']}, Star1: {row['Star1']}, Star2: {row['Star2']}, Star3: {row['Star3']}, Star4: {row['Star4']}, No of Votes: {row['No_of_Votes']}, Gross: {row['Gross']}") for _, row in df.iterrows()]

documents

[Document(metadata={'title': 'The Shawshank Redemption'}, page_content='Released Year: 1994, Certificate: A, Runtime: 142 min, Genre: Drama, IMDB Rating: 9.3, Overview: Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency., Meta score: 80.0, Director: Frank Darabont, Star1: Tim Robbins, Star2: Morgan Freeman, Star3: Bob Gunton, Star4: William Sadler, No of Votes: 2343110, Gross: 28,341,469'),
 Document(metadata={'title': 'The Godfather'}, page_content="Released Year: 1972, Certificate: A, Runtime: 175 min, Genre: Crime, Drama, IMDB Rating: 9.2, Overview: An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son., Meta score: 100.0, Director: Francis Ford Coppola, Star1: Marlon Brando, Star2: Al Pacino, Star3: James Caan, Star4: Diane Keaton, No of Votes: 1620367, Gross: 134,966,411"),
 Document(metadata={'title': 'The Dark Knight'}, page_content='Released Year: 2008, Cer

In [4]:
# Generate embeddings
embeddings = HuggingFaceBgeEmbeddings()
vector_store = FAISS.from_documents(documents, embeddings)

# Save the FAISS index
vector_store.save_local("faiss_index")

C:\Users\saisu\AppData\Local\Temp\ipykernel_3400\1136455556.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceBgeEmbeddings()
C:\Users\saisu\AppData\Local\Temp\ipykernel_3400\1136455556.py:2: LangChainDeprecationWarning: Default values for HuggingFaceBgeEmbeddings.model_name were deprecated in LangChain 0.2.5 and will be removed in 0.4.0. Explicitly pass a model_name to the HuggingFaceBgeEmbeddings constructor instead.
  embeddings = HuggingFaceBgeEmbeddings()
c:\Users\saisu\Documents\Learning\RAG_project_imdb\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See 

In [5]:
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

llm=ChatGroq(temperature=0.5, model="llama-3.2-11b-vision-preview", max_tokens=8000)

# Define the query processing prompt
query_template = PromptTemplate(
    input_variables=["query"],
    template="Extract the movie filters (genre, actor, keywords) from this query: {query}"
)

query_chain = LLMChain(llm=llm, prompt=query_template)

def extract_filters(user_query):
    response = query_chain.run(user_query)
    return response

extract_filters("Who are stars in Schindler's List")

C:\Users\saisu\AppData\Local\Temp\ipykernel_3400\248792997.py:18: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  query_chain = LLMChain(llm=llm, prompt=query_template)
C:\Users\saisu\AppData\Local\Temp\ipykernel_3400\248792997.py:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = query_chain.run(user_query)


'Unfortunately, I don\'t see any query provided. However, I can provide you with the movie filters and the list of stars from the movie "Schindler\'s List":\n\n**Movie Filters:**\n- Genre: Drama, History, War\n- Actor:\n- Keywords: Holocaust, Oskar Schindler, World War II, historical drama\n\n**Stars in Schindler\'s List:**\n1. Liam Neeson\n2. Ben Kingsley\n3. Ralph Fiennes\n4. Caroline Goodall\n5. Jonathan Sagall\n6. Emília Waikauskaitė\n7. Mark Ivanir\n8. Golshifteh Farahani'

# To JSON

In [1]:
import pandas as pd

df=pd.read_csv("C:/Users/saisu/Documents/Learning/RAG_project_imdb/data/imdb_top_1000.csv")

In [2]:
df.head(1)


,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"


In [8]:
df.duplicated().sum()

np.int64(0)

In [7]:
df["Series_Title"].duplicated().sum()

np.int64(1)

In [10]:
import json 

json_data = df.to_dict(orient="index")

# Save to a JSON file (optional)
with open("imdb_data.json", "w", encoding="utf-8") as f:
    json.dump(json_data, f, indent=4, ensure_ascii=False)

# Print sample output
print(json.dumps(json_data, indent=4, ensure_ascii=False))

{
    "0": {
        "Poster_Link": "https://m.media-amazon.com/images/M/MV5BMDFkYTc0MGEtZmNhMC00ZDIzLWFmNTEtODM1ZmRlYWMwMWFmXkEyXkFqcGdeQXVyMTMxODk2OTU@._V1_UX67_CR0,0,67,98_AL_.jpg",
        "Series_Title": "The Shawshank Redemption",
        "Released_Year": "1994",
        "Certificate": "A",
        "Runtime": "142 min",
        "Genre": "Drama",
        "IMDB_Rating": 9.3,
        "Overview": "Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.",
        "Meta_score": 80.0,
        "Director": "Frank Darabont",
        "Star1": "Tim Robbins",
        "Star2": "Morgan Freeman",
        "Star3": "Bob Gunton",
        "Star4": "William Sadler",
        "No_of_Votes": 2343110,
        "Gross": "28,341,469"
    },
    "1": {
        "Poster_Link": "https://m.media-amazon.com/images/M/MV5BM2MyNjYxNmUtYTAwNi00MTYxLWJmNWYtYzZlODY3ZTk3OTFlXkEyXkFqcGdeQXVyNzkwMjQ5NzM@._V1_UY98_CR1,0,67,98_AL_.jpg",
        "Series_Title": "